## 处理hf下载的数据集

In [165]:
import pandas as pd
import json
from opencc import OpenCC

In [166]:
train = pd.read_parquet("./datasets/steins_gate_1k/data/train-00000-of-00001.parquet")
test = pd.read_parquet("./datasets/steins_gate_1k/data/test-00000-of-00001.parquet")

In [167]:
data = pd.concat([train, test])
data

,text
0,<s>[INST] ⋯⋯啊 。[/INST] 沒錯。就在1個小時前。2010年8月13日的下...
1,<s>[INST] 這樣......這樣實在是太過分了......！ [/INST] 岡部....
2,<s>[INST] 等等。這樣不就沒辦法繼續探討了嗎。多世界詮釋什麼的怎樣都好，應該考慮我們...
3,<s>[INST] 父親在日本吧?有見過他一次嗎? [/INST] 沒有......結果都沒...
4,<s>[INST] 紅莉栖...... [/INST] 啊啊，真是的，感覺真冷啊 。</s>
...,...
90,<s>[INST] 我馬上就發送D-Mail 。[/INST] 哦，對啊，世界線改變了的話這...
91,<s>[INST] 在意嗎? [/INST] ⋯⋯⋯⋯ 。</s>
92,<s>[INST] 要解讀那個資料庫的訊息，必須同樣使用IBN5100才行 。[/INST]...
93,<s>[INST] SERN，以及控制世界的支配結構 [/INST] 你真是個幸福的傢伙啊 ...


In [169]:
converter = OpenCC("t2s")
data["text"] = data["text"].apply(converter.convert)

In [170]:
data = data["text"].apply(lambda x: pd.Series(x.replace("<s>[INST]", "").replace("</s>", "").split("[/INST]"), index=["instruction", "output"]))
data["input"] = ""

In [171]:
data

,instruction,output,input
0,⋯⋯啊 。,没错。就在1个小时前。2010年8月13日的下午2点左右 。,
1,这样......这样实在是太过分了......！,冈部...... 。,
2,等等。这样不就没办法继续探讨了吗。多世界诠释什么的怎样都好，应该考虑我们能做的事情 。,时间跳跃是无法改变结果的。刚才不说过了么 ？,
3,父亲在日本吧?有见过他一次吗?,没有......结果都没见到他 。,
4,红莉栖......,啊啊，真是的，感觉真冷啊 。,
...,...,...,...
90,我马上就发送D-Mail 。,哦，对啊，世界线改变了的话这伤口就没了......吗......,
91,在意吗?,⋯⋯⋯⋯ 。,
92,要解读那个资料库的讯息，必须同样使用IBN5100才行 。,你怎么知道啊?,
93,SERN，以及控制世界的支配结构,你真是个幸福的家伙啊 。,


In [172]:
records = data[["instruction", "input", "output"]].to_dict(orient="records")
records

[{'instruction': ' ⋯⋯啊 。',
  'input': '',
  'output': ' 没错。就在1个小时前。2010年8月13日的下午2点左右 。'},
 {'instruction': ' 这样......这样实在是太过分了......！ ',
  'input': '',
  'output': ' 冈部...... 。'},
 {'instruction': ' 等等。这样不就没办法继续探讨了吗。多世界诠释什么的怎样都好，应该考虑我们能做的事情 。',
  'input': '',
  'output': ' 时间跳跃是无法改变结果的。刚才不说过了么 ？'},
 {'instruction': ' 父亲在日本吧?有见过他一次吗? ',
  'input': '',
  'output': ' 没有......结果都没见到他 。'},
 {'instruction': ' 红莉栖...... ', 'input': '', 'output': ' 啊啊，真是的，感觉真冷啊 。'},
 {'instruction': ' 为什么问这个? ',
  'input': '',
  'output': ' 啊......不，随便问问而已！ 并没有什么特别的意思 。'},
 {'instruction': ' 我在秋叶原还有必须要做的事 。', 'input': '', 'output': ' 什么必须要做的事啊 ?'},
 {'instruction': ' 你干啥，真是烦人的家伙 。',
  'input': '',
  'output': ' 我说你啊，别向前走呀。不然我就必须倒退著走路了啊。给我横著走 。'},
 {'instruction': ' 说的关于如何处理这台时间机器 。', 'input': '', 'output': ' ⋯⋯⋯⋯ 。'},
 {'instruction': ' 喂！ 没事吧！ ', 'input': '', 'output': ' 等......呀啊啊！冈部你进来干什么！ '},
 {'instruction': ' ⋯⋯我知道的 。', 'input': '', 'output': ' 那就这样，我要走了 。'},
 {'instruction': ' 你刚才说什么！? ', 'input': '', '

In [174]:
with open("./data/hf_data.json", "w", encoding="utf-8") as f:
    json.dump(records, f, ensure_ascii=False, indent=2)